# This Notebook is an attempt at experimenting with SVG file form. 

## Goal 1. Open an .svg file and extract the paths

## Goal 2. Convert the paths into meaningful points, tracking position after each command relative to the origin

## Goal 3. Create robot commands 

Before trying to run this make sure that you have Beautiful Soup and Python3 running in your interpreter

In [1]:
from bs4 import BeautifulSoup as bs # This is my personal favorite library for HTML and XML in Python
import re
#import numpy as np #might be useful latter
import math

# The next line opens file '76star.svg' of type 'xml'
soup = bs(open('76star.svg'), 'xml')
print(soup.prettify()) #bs.prettify prints a nicely formatted string, but soup is a tree

print('\n\n')
svg = soup.svg # get the svg info from soup

# finally we can find all of the path elements that are children of svg, and subsequently get the d elements iterating 
# through
for path in svg.find_all('path'): 
    print(path.get('d'))

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html>
<html>
 <body>
  <svg height="932.000000pt" preserveAspectRatio="xMidYMid meet" version="1.0" viewBox="0 0 1400.000000 932.000000" width="1400.000000pt" xmlns="http://www.w3.org/2000/svg">
   <g fill="#000000" stroke="none" transform="translate(0.000000,932.000000) scale(0.100000,-0.100000)">
    <path d="M8114 278 c-9 -42 -10 -43 -48 -40 l-39 3 31 -35 c30 -35 30 -35 17 -85 l-13 -50 30 22 30 22 25 -30 25 -30 -6 46 c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11z"/>
   </g>
   Sorry, your browser does not support inline SVG.
  </svg>
 </body>
</html>



M8114 278 c-9 -42 -10 -43 -48 -40 l-39 3 31 -35 c30 -35 30 -35 17 -85 l-13 -50 30 22 30 22 25 -30 25 -30 -6 46 c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11z



**Hopefully**, we successfully opened the '76star.svg' file and printed the data of the <path> tags. The first print statement should show the original file we worked with, while the second should show our targetted <path> data. It may not look like much, but this is nearly all we need. Unfortunately, the <path> data is not overwhelmingly beautiful, like our xml file. Below is a little more info to understand what we're really looking at



## SVG Path Info
#### source: https://css-tricks.com/svg-path-syntax-illustrated-guide/

> The <path> element is used to define a path.

> The following commands are available for path data:
```
    M = moveto
    L = lineto
    H = horizontal lineto
    V = vertical lineto
    C = curveto
    S = smooth curveto
    Q = quadratic Bézier curve
    T = smooth quadratic Bézier curveto
    A = elliptical Arc
    Z = closepath
```
>Note: All of the commands above can also be expressed with lower letters. Capital letters means absolutely positioned, lower cases means relatively positioned.

Those look like good candidates for python functions, if I've ever seen any. At this juncture, the two essential commands look to be moveto, lineto, curveto, and finally closepath. They'll probably need to take parameters, like points, cur_point, absolute=True/False. That's a problem for later though. Right now we need to get our data properly formatted. Adding some whitespace we can more easily see each individual command from our prior example 

M8114 278 

c-9 -42 -10 -43 -48 -40 

l-39 3 31 -35 

c30 -35 30 -35 17 -85 

l-13 -50 30 22 30 22 25 -30 25 -30 -6 46 

c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11

z

*which really means...*

**Moveto** (8114, 278) (the absolute point)

**curveto** (-9, -42) (-10, -43) (-48, -40) (relative to the current point)

**lineto** (-39, 3) (31, -35) (relative to the current point)

**curveto** (30, -35) (30, -35) (17, -85) (relative to the current point)

**lineto** (-13, -50) (30, 22) (30, 22) (25, -30) (25, -30) (-6, 46) (relative to the current point)

**curveto** (-4, 40) (-2, 49) (19, 69) (32, 30) (31, 34) (-7, 45) (-29, 9) (-34, 15) (-39, 52) (-7, 58) (-14, 61) (-25, 11) (relative to the current point)

**close path** (back to the absolute point 8114 278)

To visualize all this look at the below code, and paste it over the example https://www.w3schools.com/graphics/tryit.asp?filename=trysvg_polygon3

It is not exactly our previous example code (which is part of a larger file) but it uses the same general curves. The first path with a pink fill is the shape that our example code draws. The subsequent paths are individual parts, which make the whole, and an additional Moveto command to properly position each of them. Observe how the red curve starts at (250,250). It then moves to the relative points (-9 -42) and (-10 -43) before ending at the relative point (-48 -40), which is the absolute point (250-48, 250-40), or (202, 210). The black line starts at the absolute point (202, 210) and thus the two lines meet there. To explore this in depth, scroll to the bottom and paste that into the above link

```
<!DOCTYPE html>
<html>
<body>

<svg height="300pt" width="600pt"> 
	<g fill="pink">
      <path stroke="none" d=
          "M250 250 
          c-9 -42 -10 -43 -48 -40 
          l-39 3 31 -35 
          c30 -35 30 -35 17 -85 
          l-13 -50 30 22 30 22 25 -30 25 -30 -6 46 
          c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11 
          z">
  	</g> 
	<g fill="none">
      <path stroke="red" d="M 250 250
   		c-9 -42 -10 -43 -48 -40 " />
      <path stroke="black" d="M 202 210 l-39 3 31 -35" />
      <path stroke="blue" d="M 194 178 c30 -35 30 -35 17 -85" />
      <path stroke="green" d="M 211 93 l-13 -50 30 22 30 22 25 -30 25 -30 -6 46" />
      <path stroke="yellow" d="M 302 73 c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11" />
  	</g>  
   Sorry, your browser does not support inline SVG.
  </svg>

</body>
</html>
```

## Putting it all together

This is great to visualize and understand the data that we are working with, but now we will need to get a little more practical and create a *function*, which can
1. open an XML file
2. extract the svg path data
3. format the data into a useful format that we can create robot commands from

As the order of the paths may be important, we should use a list to store them. Each path can be further split into two distinct parts, *command* and *points*, so a tuple is natural here. As the order of the points, must also be preserved, we should store them in a list as well. Naturally a point is two immutable elements, so that will also be a tuple. Pause. This is a lot to take in

Below is a visual to aid in percieving our new structure
```
[('M', [(8114, 278)])
('c', [(-42, -10), (-43, -48)])
('l', [(-40, 3), (31, -35)])
('c', [(30, -35), (30, -35), (17, -85)])
('l', [(-50, 30), (22, 30), (22, 25), (-30, 25), (-30, -6)])
('c', [(46, 40), (-2, 49), (19, 69), (32, 30), (31, 34), (-7, 45), (-29, 9), (-34, 15), (-39, 52), (-7, 58), (-14, 61)])
('z', [])]
```

It's a list of paths.

Each path is tuple of a command and list points.

Each point is a tuple

In [2]:
def svg_to_paths(file_name):
    '''
    This method takes one parameter, an xml file_name. It extracts svg tags, and returns and list containing a tuple
    with a command and a list of points, which are also tuples.
    [(command, [(point1x, point1y), (point2x, point2y), ..., (pointNx, pointNy)]),]
    
    it's just lists of tuples all the way down
    '''
    #load file_name and get svg tag
    soup = bs(open(file_name), 'xml')
    svg = soup.svg
    
    commands = {
            'M' : 'moveto',
            'L' : 'lineto',
            'H' : 'hlineto',
            'V' : 'vertical lineto',
            'C' : 'curveto',
            'S' : 'smooth curveto',
            'Q' : 'quadratic Bézier curve',
            'T' : 'smooth quadratic Bézier curveto',
            'A' : 'elliptical Arc',
            'Z' : 'closepath'
            }
    paths = []
    #pat = re.compile(r'[.]+')
    for path in svg.find_all('path'):
        d = path.get('d')
        d = d.split() # we got the data from path and tokenized it, splitting on white space
        
        # now we're going to iterate through the tokens, as follows
        # this is a pretty hack-y solution, but it works, so I'm rolling with it. 
        # please feel free to improve this, but if you break it, you bought it
        points = []
        command = ''
        x = 0
        y = 0
        i = 0 
        for tok in d:
            if tok.isdigit(): # no alpha char, so we know there's no command in this token
                if i%2 != 0: # this is the most screwey part, to pair the numbers into 
                    y = int(tok)
                    points.append((x,y))
                else:
                    x = int(tok)
            elif tok[0] == '-' and tok[1:].isdigit():
                if i%2 != 0:
                    y = int(tok[1:])*(-1)
                    points.append((x,y))
                else:
                    x = int(tok[1:])*(-1)
            elif tok[0].isalpha():
                tup = (command, points)
                paths.append(tup)
                command = tok[0]
                points = []
                if tok[1:].isdigit():
                    x = int(tok[1:])
                else: # there was no point on this iteration, so we correct i to keep x,y rhythm 
                    i-=1
            elif tok.endswith('Z') or tok.endswith('z'):
                tup = (command, points)
                paths.append(tup)
                paths.append(('z',[]))
            else:
                print(tok)
            i+=1
        for path in paths:
            if path[0] == '':
                paths.remove(path)
    return paths

triangle = svg_to_paths('76star.svg')
print('If our function works as expected, this output should resemble the format above')
for each in triangle:
    print(each)

If our function works as expected, this output should resemble the format above
('M', [(8114, 278)])
('c', [(-42, -10), (-43, -48)])
('l', [(-40, 3), (31, -35)])
('c', [(30, -35), (30, -35), (17, -85)])
('l', [(-50, 30), (22, 30), (22, 25), (-30, 25), (-30, -6)])
('c', [(46, 40), (-2, 49), (19, 69), (32, 30), (31, 34), (-7, 45), (-29, 9), (-34, 15), (-39, 52), (-7, 58), (-14, 61)])
('z', [])


## Goal #2
#### Convert the paths into meaningful points, tracking position after each command relative to the origin

In [3]:
'''
    Store location (x and y) and angle of robot. May add speed at some point
'''
class State:
    
    '''
        Initialize robot at Origin
        Origin is top left corner of paper with Robot pointing down. Down and Right are positive
        Why it's this way, I can't say
        SVG just liked it better that way
    '''
    def __init__(self, x=0.0, y=0.0, angle=0.0):
        self.x = x
        self.y = y
        self.angle = angle

## Goal #3
#### Create robot commands

> The following commands are available for path data:
```
    M = moveto
    L = lineto
    H = horizontal lineto
    V = vertical lineto
    C = curveto
    S = smooth curveto
    Q = quadratic Bézier curve
    T = smooth quadratic Bézier curveto
    A = elliptical Arc
    Z = closepath
```
>Note: All of the commands above can also be expressed with lower letters. Capital letters means absolutely positioned, lower cases means relatively positioned.

Those look like good candidates for python functions, if I've ever seen any. At this juncture, the two essential commands look to be moveto, lineto, curveto, and finally closepath. They'll probably need to take parameters, like points, cur_point, absolute=True/False. That's a problem for later though. Right now we need to get our data properly formatted. Adding some whitespace we can more easily see each individual command from our prior example 

In [4]:
''' Scratch Paper - figure your shit out'''

# determine angle and distance (duration * speed) e.i. cartesian coords to polar coords
# r = (x2 + y2)^1/2  
# θ = atan(y / x)  

# Getting that angle
point = (2 , 2)
state = State()
vector = {}

vector['x'] = point[0] - state.x
vector['y'] = point[1] - state.y
# determine angle and distance (duration * speed) e.i. cartesian coords to polar coords
# r = (x2 + y2)^1/2  
r = math.atan2(vector['y'], vector['x'])

print('arctan(', vector['y'], '/' , vector['x'] ,') = ', r , ' or ', math.degrees(r), ' degrees') 

def get_theta(x1, y1, x2, y2):
    '''
        takes four arguments x1, y1, x2, y2
        returns the angle in radians of arctan(y2-y1, x2-x1)
        
        Use this to convert from cartesian coordinates to polar coordinates
    '''
    r = math.atan2(y2-y1, x2-x1)
    return r

print('Our method says hopefully the same thing: ', get_theta(state.x, state.y, point[0], point[1]))

def get_distance(x1, y1, x2, y2):
    '''
        takes four arguments x1, y1, x2, y2
        returns the distance between the points (in no units or maybe all units)
        
        Use this to convert from cartesian coordinates to polar coordinates
    '''
    x, y = x2 - x1, y2-y1
    return(math.sqrt(math.pow(x, 2) + math.pow(y, 2)))
    
print('This should give the distance: ', get_distance(state.x, state.y, point[0], point[1]))

def cart_to_polar(x1, y1, x2, y2):
    '''
        takes four arguments x1, y1, x2, y2
        returns a dictionary with the angle in radians and distance between the points 
        {'dist': 2.8284271247461903, 'theta': 0.7853981633974483}
        
        Use this to convert from cartesian coordinates to polar coordinates
    '''
    return({'theta': get_theta(x1, y1, x2, y2), 'dist': get_distance(x1, y1, x2, y2)})
    
cart_to_polar(state.x, state.y, point[0], point[1])

arctan( 2.0 / 2.0 ) =  0.7853981633974483  or  45.0  degrees
Our method says hopefully the same thing:  0.7853981633974483
This should give the distance:  2.8284271247461903


{'dist': 2.8284271247461903, 'theta': 0.7853981633974483}

In [5]:
def rotate(theta):
    ''' 
        this is going to be the place where we calibrate our robot, and we enter the real world, translating radians to
        1. direction of rotation (a simple if statement to turn the shortest way)
        2. speed of rotation (This should be fixed)
        3. duration of rotation 
    '''
    pass

def go(distance):
    ''' 
        this is going to be the place where we calibrate our robot, and we enter the real world.
        Only two questions here. 
        1. How fast do we go?
        2. For how long?
    '''
    pass
    
def moveto(state, point, absolute=True):
    '''
        Takes three arguments, state, point, and absolute=True
        point should be a list of points or preferrably a single point
        currently moveto only supports absolute=True
    '''
    if point is list:
        point = point[1]
    elif point is not tuple:
        print('Oh Fuck, we got an unhandle error. Buckle up for some bouncy unpredicted behavior and probably a crash')
        print('Haha JK, we are just going to ignore that shit')
        return state
    # get vector in polar cordinates
    polar = cart_to_polar(state.x, state.y, point[0],point[1])
    # rotate angle
    rotate(polar['theta'])
    # go to there
    go(polar['distance'])
    
    # update and return? state, now at new point and angled in direction just travelled 
    state.x = point[0]
    state.y = point[1]
    state.angle =  polar['theta']
    return state
    
def lineto(state, points, absolute=False):
    '''
    Takes three arguments, state, point, and absolute=False
    point should be a list of points with at least one point
    '''
    if points is not list and points is not tuple:
        print('Oh Fuck, we got an unhandle error. Buckle up for some bouncy unpredicted behavior and probably a crash')
        print('Haha JK, we are just going to ignore that shit')
        return state
    
    if absolute == False:
            for point in points:
                # get vector in polar cordinates
                polar = cart_to_polar(0, 0, point[0],point[1])
                # rotate angle
                rotate(polar['theta'])
                # go to there
                go(polar['distance'])

                # update state, now at new point and angled in direction just traveled. 
                state.x = point[0]
                state.y = point[1]
                state.angle =  polar['theta']
    
    # same shit as moveto really, but loop for each point, updating status
    for point in points:
        # get vector in polar cordinates
        polar = cart_to_polar(state.x, state.y, point[0],point[1])
        # rotate angle
        rotate(polar['theta'])
        # go to there
        go(polar['distance'])
        
        # update state, now at new point and angled in direction just traveled. 
        state.x = point[0]
        state.y = point[1]
        state.angle =  polar['theta']
        
        
    return state


## I didn't think anyone would actually get this far

from here on out this is very much a work in progress and nothing is likely to work until I get further

Really the only part of this that is anywhere near complete is the Goal 1 portion, but it's a start

In [6]:
def execute(file_name):
    state = State()
    paths = svg_to_paths(file_name)
    for path in paths:
        #print(path)
        for command in path[:5]:
            print(command)
            '''
            if command[0] == 'M':
                moveto(state, command[0], absolute=True)
            elif command[0] == 'm':
                moveto(state, command[0], absolute=False) # this shit might not work
            elif command[0] == 'L':
                lineto(state, command[0], absolute=True)
            elif command[0] == 'l':
                lineto(state, command[0], absolute=False)
            elif command[0] == 'C':
                print('unsupported command: ', command[0])
            elif command[0] == 'c':
                print('unsupported command: ', command[0])
            else:
                print('unsupported command: ', command[0])
            '''
        
execute('jojo.svg')

M
[(0, 4660)]
l
[(0, -4660), (2575, 0), (2575, 0), (-5, 53)]
c
[(28, -7), (79, -9), (112, -2), (33, -13), (96, -25), (140, -27), (109, -59), (286, -67), (381, -5), (61, -12), (85, -29), (104, -39), (45, -65), (88, -65), (109, 0), (27, 13), (27, 25), (-1, 6), (-12, 15), (-23, 20), (-24, 6), (-2, 31), (25, 58), (59, 26), (34, 88), (104, 138), (156, 198), (205, 266), (295, 383), (509, 85), (154, 132), (263, 184), (422, 104), (322, 145), (705, 84), (781, -17), (20, -18), (20, -14), (-20, 3), (-34, 1), (-41, -13), (-41, -16), (0, -17), (-8, -12), (-73, 6), (-71, -3), (-160, -24), (-239, -7), (-28, -15), (-38, -29), (-38, -11), (0, -20), (6, -20), (13, 0), (6, -12), (17, -26), (24, -24), (11, -26), (10, -21), (-7, 3), (-11, 1), (-20, -4), (-20, -4), (0, -11), (22, -15), (49, -6), (44, -9), (50, -35), (55, -21), (4, -27), (10, -23), (21, 5), (12, 2), (14, -12), (8, -11), (-3, -22), (-11, -25), (-16, -4), (-5, -12), (-6, -19), (-2, -8), (5, 1), (15, 24), (31, 37), (25, 48), (54, 21), (54, -16)

In [7]:
# This is here so I can compare it to what I'm getting up top
soup = bs(open('jojo.svg'), 'xml')
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 20010904//EN" "http://www.w3.org/TR/2001/REC-SVG-20010904/DTD/svg10.dtd">
<svg height="932.000000pt" preserveAspectRatio="xMidYMid meet" version="1.0" viewBox="0 0 1400.000000 932.000000" width="1400.000000pt" xmlns="http://www.w3.org/2000/svg">
 <metadata>
  Created by potrace 1.15, written by Peter Selinger 2001-2017
 </metadata>
 <g fill="#000000" stroke="none" transform="translate(0.000000,932.000000) scale(0.100000,-0.100000)">
  <path d="M0 4660 l0 -4660 2575 0 2575 0 -5 53 c-3 28 -7 79 -9 112 -2 33 -13 96 -25 140 -27 109 -59 286 -67 381 -5 61 -12 85 -29 104 -39 45 -65 88 -65 109 0 27 13 27 25 -1 6 -12 15 -23 20 -24 6 -2 31 25 58 59 26 34 88 104 138 156 198 205 266 295 383 509 85 154 132 263 184 422 104 322 145 705 84 781 -17 20 -18 20 -14 -20 3 -34 1 -41 -13 -41 -16 0 -17 -8 -12 -73 6 -71 -3 -160 -24 -239 -7 -28 -15 -38 -29 -38 -11 0 -20 6 -20 13 0 6 -12 17 -26 24 -24 11 -26 10 -21 -7 3 -11 1 -20 -4 -20

## Explore .svg More
#### paste this code into https://www.w3schools.com/graphics/tryit.asp?filename=trysvg_polygon3

```
<!DOCTYPE html>
<html>
<body>

<svg height="300pt" width="600pt"> 

     <!-- This is the example shape @(250,250) instead of (8114, 278) -->
	<g fill="pink">
      <path stroke="none" d="M250 250 c-9 -42 -10 -43 -48 -40 l-39 3 31 -35 c30 -35 30 -35 17 -85 l-13 -50 30 22 30 22 25 -30 25 -30 -6 46 c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11 z"/>
  	</g> 
    
    <!-- These are the individual part making the shape -->
	<g fill="none">
      <path stroke="red" d="M 250 250
   		c-9 -42 -10 -43 -48 -40 " />
      <path stroke="black" d="M 202 210 l-39 3 31 -35" />
      <path stroke="blue" d="M 194 178 c30 -35 30 -35 17 -85" />
      <path stroke="green" d="M 211 93 l-13 -50 30 22 30 22 25 -30 25 -30 -6 46" />
      <path stroke="yellow" d="M 302 73 c-4 40 -2 49 19 69 32 30 31 34 -7 45 -29 9 -34 15 -39 52 -7 58 -14 61 -25 11" />
  	</g>  
    
     <!-- Mark relevant points -->
     
     <!-- Below we can observe how curveto and lineto frustratingly differ-->
     <!-- lineto moves relative to the previous point in the command-->
     
     <!-- curveto has two type of points, which I'll call anchors and control -->
     <!-- the first and last points in curve to are anchors -->
     <!-- controls moves relative to the most recent anchor-->
     <!-- there seem to alway be two -->
     <!-- then there is another anchor, which also moves relative to the last anchor-->
     
     
    <g fill="red"> <!-- red curveto points -->
      <circle id="A" cx="250" cy="250" r="2" />
      <!-- NOTE: These are control points -->
      <circle fill="white" stroke="red" id="B" cx="241" cy="209" r="2" />
      <circle fill="white" stroke="red" id="C" cx="240" cy="207" r="2" />
      <circle id="D" cx="202" cy="210" r="4" />
    </g>
    <g fill="Black"> <!-- black lineto points -->
      <circle id="A" cx="202" cy="210" r="2" />
      <circle id="B" cx="163" cy="213" r="2" />
      <circle id="C" cx="194" cy="178" r="4" />
    </g>
    <g fill="Blue"> <!-- blue curve to points -->
      <circle id="A" cx="194" cy="178" r="2" />
      <!-- NOTE: These are control points at the same location-->
      <circle fill="white" stroke="blue" id="B" cx="224" cy="143" r="2" />
      <circle fill="white" stroke="blue" id="C" cx="224" cy="143" r="2" />
      <circle id="D" cx="211" cy="93" r="4" />
    </g>
    <g fill="green"> <!-- green lineto points -->
      <circle id="A" cx="211" cy="93" r="2" />
      <circle id="B" cx="198" cy="43" r="2" /> <!---13 -50-->
      <circle id="C" cx="228" cy="65" r="2" /> <!--30 22-->
      <circle id="D" cx="258" cy="87" r="2" /> <!-- 30 22 -->
      <circle id="E" cx="283" cy="57" r="2" /> <!--25 -30 --> 
      <circle id="F" cx="308" cy="27" r="2" /> <!-- 25 -30 --> 
      <circle id="G" cx="302" cy="73" r="4" /> <!-- -6 46 -->
    </g>
    <g fill="Yellow"> <!-- yellow curve to points -->
      <circle id="A" cx="302" cy="73" r="2" />
      <!--  Two control points -->
      <circle fill="black" stroke="yellow" id="B" cx="298" cy="113" r="2" />
      <circle fill="black" stroke="yellow" id="C" cx="300" cy="122" r="2" />
      <!-- new anchor -->
      <circle fill="yellow" id="D" cx="321" cy="142" r="2" />
      <!--  Two more control points -->
      <circle fill="black" stroke="yellow" id="E" cx="353" cy="172" r="2" />
      <circle fill="black" stroke="yellow" id="F" cx="352" cy="176" r="2" />
      <!-- new anchor -->
      <circle fill="yellow" id="D" cx="314" cy="187" r="2" />
      <!--  Two more control points -->
      <circle fill="black" stroke="yellow" id="E" cx="285" cy="198" r="2" />
      <circle fill="black" stroke="yellow" id="F" cx="280" cy="202" r="2" />
      <!-- new anchor -->
      <circle fill="yellow" id="D" cx="275" cy="239" r="2" />
      <!--  Two more control points -->
      <circle fill="black" stroke="yellow" id="E" cx="268" cy="297" r="2" />
      <circle fill="black" stroke="yellow" id="F" cx="261" cy="300" r="2" />
      <!--  Final Anchor / End Point / Closes loop -->
      <circle id="x" cx="250" cy="250" r="4" />
      
      <!-- -7 58 -14 61 -25 11 -->
    </g>
   Sorry, your browser does not support inline SVG.
  </svg>

</body>
</html>
```